In [3]:
# AI Generated using Claude

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random
import math

# Set random seed for reproducibility
np.random.seed(2)
random.seed(2)

# Historical quarterly revenue data (in millions) - baseline targets
quarterly_targets = {
    '2019 Q4': 106, '2020 Q1': 122, '2020 Q2': 146, '2020 Q3': 167, '2020 Q4': 188,
    '2021 Q1': 328, '2021 Q2': 664, '2021 Q3': 777, '2021 Q4': 882,
    '2022 Q1': 956, '2022 Q2': 1021, '2022 Q3': 1051, '2022 Q4': 1071,
    '2023 Q1': 1074, '2023 Q2': 1099, '2023 Q3': 1102, '2023 Q4': 1118,
    '2024 Q1': 1105, '2024 Q2': 1139, '2024 Q3': 1137, '2024 Q4': 1146,
    '2025 Q1': 1141, '2025 Q2': 1163, '2025 Q3': 1178
}

# Subscription Type Distribution by Time Period
SUBSCRIPTION_DISTRIBUTION = {
    # Early periods (2019-2021): More yearly subscriptions
    'early': {
        'Monthly': 8,
        'Yearly': 92
    },
    # Mid periods (2022-2023): Transition period
    'mid': {
        'Monthly': 12,
        'Yearly': 88
    },
    # Recent periods (2024-2025): More monthly adoption
    'recent': {
        'Monthly': 18,
        'Yearly': 82
    }
}

# Regional Distribution by Time Period
REGIONAL_DISTRIBUTION = {
    # Early periods (2019-2021)
    'early': {
        'NA': 55,
        'EMEA': 30,
        'APAC': 10,
        'LATAM': 5
    },
    # Mid periods (2022-2023)
    'mid': {
        'NA': 48,
        'EMEA': 35,
        'APAC': 12,
        'LATAM': 5
    },
    # Recent periods (2024-2025)
    'recent': {
        'NA': 40,
        'EMEA': 35,
        'APAC': 18,
        'LATAM': 7
    }
}

# Product Distribution by Region and Time Period
PRODUCT_DISTRIBUTION = {
    # Early periods (2019-2021)
    'early': {
        'NA': {'Pro': 10, 'Business': 40, 'Enterprise': 50},
        'EMEA': {'Pro': 20, 'Business': 40, 'Enterprise': 40},
        'APAC': {'Pro': 25, 'Business': 45, 'Enterprise': 30},
        'LATAM': {'Pro': 30, 'Business': 50, 'Enterprise': 20}
    },
    # Mid periods (2022-2023)
    'mid': {
        'NA': {'Pro': 20, 'Business': 45, 'Enterprise': 35},
        'EMEA': {'Pro': 20, 'Business': 60, 'Enterprise': 20},
        'APAC': {'Pro': 40, 'Business': 55, 'Enterprise': 5},
        'LATAM': {'Pro': 35, 'Business': 55, 'Enterprise': 10}
    },
    # Recent periods (2024-2025)
    'recent': {
        'NA': {'Pro': 40, 'Business': 35, 'Enterprise': 25},
        'EMEA': {'Pro': 35, 'Business': 45, 'Enterprise': 20},
        'APAC': {'Pro': 35, 'Business': 30, 'Enterprise': 35},
        'LATAM': {'Pro': 45, 'Business': 45, 'Enterprise': 10}
    }
}

regions = ['NA', 'EMEA', 'APAC', 'LATAM']
products = ['Pro', 'Business', 'Enterprise']

def get_time_period(year):
    """Determine time period for product distribution"""
    if year <= 2021:
        return 'early'
    elif year <= 2023:
        return 'mid'
    else:
        return 'recent'

def get_randomized_subscription_split(time_period):
    """Get randomized subscription split based on time period baseline with variation"""
    base_monthly = SUBSCRIPTION_DISTRIBUTION[time_period]['Monthly']
    
    # Add randomization: ±3% variation from base
    variation = random.uniform(-3, 3)
    monthly_pct = max(1, min(99, base_monthly + variation))  # Keep between 1-99%
    yearly_pct = 100 - monthly_pct
    
    return monthly_pct / 100, yearly_pct / 100

def parse_quarter(quarter_str):
    """Parse quarter string to get year and quarter number"""
    parts = quarter_str.split(' ')
    year = int(parts[0])
    quarter_num = int(parts[1][1])
    return year, quarter_num

def get_quarter_dates(year, quarter_num):
    """Get start and end dates for fiscal quarter"""
    # Fiscal year: Feb-Jan (Q1: Feb-Apr, Q2: May-Jul, Q3: Aug-Oct, Q4: Nov-Jan)
    if quarter_num == 1:  # Feb 1 - Apr 30
        start_date = datetime(year, 2, 1)
        end_date = datetime(year, 4, 30)
    elif quarter_num == 2:  # May 1 - Jul 31
        start_date = datetime(year, 5, 1)
        end_date = datetime(year, 7, 31)
    elif quarter_num == 3:  # Aug 1 - Oct 31
        start_date = datetime(year, 8, 1)
        end_date = datetime(year, 10, 31)
    else:  # Nov 1 - Jan 31 (next year)
        if quarter_num == 4:
            start_date = datetime(year, 11, 1)
            end_date = datetime(year + 1, 1, 31)
        
    return start_date, end_date

class TrendGenerator:
    """Generate trends with randomization"""
    
    def __init__(self, total_weeks):
        self.total_weeks = total_weeks
        self.trends = {}
        self.generate_patterns()
    
    def generate_patterns(self):
        """Generate patterns for each region-product combination"""
        for region in regions:
            for product in products:
                key = f"{region}_{product}"
                self.trends[key] = self.create_trend_sequence()
    
    def create_trend_sequence(self):
        """Create a sequence of more realistic trend phases with better variation"""
        phases = []
        current_week = 0
        
        while current_week < self.total_weeks:
            # Trends last between 3 and 16 weeks
            phase_length = random.randint(3, 16)
            phase_length = min(phase_length, self.total_weeks - current_week)
            
            trend_types = ['steady_growth', 'gradual_decline', 'seasonal_wave', 
                          'slow_recovery', 'plateau', 'gentle_acceleration',
                          'market_correction', 'growth_spurt', 'sideways_drift',
                          'volatile_growth', 'cyclical_pattern']
            
            trend_type = random.choice(trend_types)
            
            # Determines strength of trend, scales revenue
            intensity = random.uniform(0.3, 2.0)
            
            # Transition between phases
            transition_factor = random.uniform(0.8, 1.2)
            
            phases.append({
                'start_week': current_week,
                'end_week': current_week + phase_length,
                'trend_type': trend_type,
                'intensity': intensity,
                'base_value': random.uniform(0.7, 1.3),
                'transition_factor': transition_factor
            })
            
            current_week += phase_length
        
        return phases
    
    def get_value(self, combination_key, week_num):
        """Get the trend value for a specific combination at a specific week"""
        phases = self.trends[combination_key]
        
        # Find the current phase
        current_phase = None
        for phase in phases:
            if phase['start_week'] <= week_num < phase['end_week']:
                current_phase = phase
                break
        
        if not current_phase:
            return 1.0
        
        # Calculate position within phase (0 to 1)
        phase_progress = (week_num - current_phase['start_week']) / max(1, current_phase['end_week'] - current_phase['start_week'])
        
        return self.calculate_trend_value(current_phase, phase_progress)
    
    def calculate_trend_value(self, phase, progress):
        """Calculate more realistic trend values with better variation"""
        base = phase['base_value']
        intensity = phase['intensity']
        transition = phase['transition_factor']
        
        # Add baseline noise to all trend types for variation
        baseline_noise = np.random.normal(0, 0.04)  # Small random variation for all trends
        
        if phase['trend_type'] == 'steady_growth':
            # Realistic business growth with some variability
            growth_factor = 1 + (intensity * 0.2 * math.log(1 + progress * 2))
            weekly_variation = np.random.normal(0, 0.06)  # Weekly business variation
            return base * growth_factor * (1 + weekly_variation + baseline_noise)
        
        elif phase['trend_type'] == 'gradual_decline':
            # Business decline with occasional upticks
            decline_factor = math.exp(-intensity * 0.15 * progress)
            uptick = 0.02 * math.sin(progress * math.pi * 6)  # Small upticks
            weekly_variation = np.random.normal(0, 0.05)
            return base * decline_factor * (1 + uptick + weekly_variation + baseline_noise)
        
        elif phase['trend_type'] == 'seasonal_wave':
            # Seasonal patterns with business variability
            wave_factor = 1 + (intensity * 0.25 * math.sin(progress * math.pi * 2))
            trend_component = 1 + (intensity * 0.1 * progress)  # Slight underlying trend
            weekly_variation = np.random.normal(0, 0.05)
            return base * wave_factor * trend_component * (1 + weekly_variation + baseline_noise)
        
        elif phase['trend_type'] == 'slow_recovery':
            # Recovery after market correction
            recovery_factor = 1 - 0.7 * math.exp(-intensity * progress * 2.5)
            weekly_variation = np.random.normal(0, 0.04)
            return base * recovery_factor * transition * (1 + weekly_variation + baseline_noise)
        
        elif phase['trend_type'] == 'plateau':
            # Business plateau with minor fluctuations
            plateau_noise = intensity * 0.08 * math.sin(progress * math.pi * 12)
            weekly_variation = np.random.normal(0, 0.06)
            return base * (1 + plateau_noise + weekly_variation + baseline_noise)
        
        elif phase['trend_type'] == 'gentle_acceleration':
            # Accelerating growth phase
            accel_factor = 1 + (intensity * 0.3 * progress * progress)
            weekly_variation = np.random.normal(0, 0.05)
            return base * accel_factor * transition * (1 + weekly_variation + baseline_noise)
        
        elif phase['trend_type'] == 'market_correction':
            # Sharp correction followed by stabilization
            if progress < 0.3:
                correction = 1 - (intensity * 0.2 * progress / 0.3)
            else:
                correction = 1 - intensity * 0.2 + (intensity * 0.1 * (progress - 0.3) / 0.7)
            weekly_variation = np.random.normal(0, 0.05)
            return base * correction * (1 + weekly_variation + baseline_noise)
        
        elif phase['trend_type'] == 'growth_spurt':
            # Rapid growth that levels off
            if progress < 0.5:
                spurt_factor = 1 + intensity * 0.4 * (progress / 0.5)
            else:
                spurt_factor = 1 + intensity * 0.4 * (1 - (progress - 0.5) * 0.5)
            weekly_variation = np.random.normal(0, 0.06)
            return base * spurt_factor * (1 + weekly_variation + baseline_noise)
        
        elif phase['trend_type'] == 'sideways_drift':
            # Sideways movement with random drift
            drift = intensity * 0.1 * (progress - 0.5)
            weekly_variation = np.random.normal(0, intensity * 0.06)
            return base * (1 + drift + weekly_variation + baseline_noise)
        
        elif phase['trend_type'] == 'volatile_growth':
            # Growth with higher volatility
            growth_trend = 1 + intensity * 0.2 * progress
            volatility = intensity * 0.15 * math.sin(progress * math.pi * 8)
            weekly_variation = np.random.normal(0, 0.07)
            return base * growth_trend * (1 + volatility + weekly_variation + baseline_noise)
        
        elif phase['trend_type'] == 'cyclical_pattern':
            # Multiple cycles within the phase
            cycles = 2 + intensity
            cycle_factor = 1 + (intensity * 0.2 * math.sin(progress * math.pi * cycles))
            weekly_variation = np.random.normal(0, 0.05)
            return base * cycle_factor * (1 + weekly_variation + baseline_noise)
        
        else:
            return base * (1 + baseline_noise)

def generate_weekly_data():
    """Generate weekly data with realistic patterns"""
    records = []
    total_week_number = 0
    
    # Calculate total weeks
    total_weeks = 0
    for quarter_str in quarterly_targets.keys():
        year, quarter_num = parse_quarter(quarter_str)
        start_date, end_date = get_quarter_dates(year, quarter_num)
        quarter_days = (end_date - start_date).days + 1
        num_weeks = math.ceil(quarter_days / 7)
        total_weeks += num_weeks
    
    # Initialize trend generator
    trend_generator = TrendGenerator(total_weeks)
    
    # Generate base data
    for quarter_str in quarterly_targets.keys():
        year, quarter_num = parse_quarter(quarter_str)
        start_date, end_date = get_quarter_dates(year, quarter_num)
        
        quarter_days = (end_date - start_date).days + 1
        num_weeks = math.ceil(quarter_days / 7)
        
        current_date = start_date
        week_in_quarter = 1
        
        # Get time period for distributions
        time_period = get_time_period(year)
        
        for week in range(num_weeks):
            week_end = min(current_date + timedelta(days=6), end_date)
            
            # Get randomized subscription split for this week
            monthly_pct, yearly_pct = get_randomized_subscription_split(time_period)
            
            # Generate data for each region-product combination
            for region in regions:
                for product in products:
                    combination_key = f"{region}_{product}"
                    
                    # Get trend value
                    trend_value = trend_generator.get_value(combination_key, total_week_number)
                    
                    # Add more realistic seasonality and randomness
                    seasonal_factor = 1 + 0.12 * math.sin((current_date.month / 12) * 2 * math.pi)
                    random_factor = np.random.uniform(0.88, 1.12)  # Weekly business variation
                    
                    # Apply regional distribution weight (now time-period aware)
                    regional_weight = REGIONAL_DISTRIBUTION[time_period][region] / 100
                    
                    # Apply product distribution weight for this region and time period
                    product_weight = PRODUCT_DISTRIBUTION[time_period][region][product] / 100
                    
                    # Base revenue (this will be scaled later)
                    base_revenue = trend_value * seasonal_factor * random_factor * regional_weight * product_weight
                    
                    # Ensure positive values
                    base_revenue = max(0.01, base_revenue)
                    
                    # Split between monthly and yearly using randomized percentages
                    monthly_revenue = base_revenue * monthly_pct
                    yearly_revenue = base_revenue * yearly_pct
                    
                    # Calculate customer metrics
                    arpu_base = {
                        'Pro': {'Monthly': 15, 'Yearly': 150},
                        'Business': {'Monthly': 240, 'Yearly': 2000},
                        'Enterprise': {'Monthly': 2400, 'Yearly': 20000}
                    }
                    
                    # Regional ARPU multipliers
                    region_multipliers = {'NA': 1.0, 'EMEA': 0.8, 'APAC': 0.7, 'LATAM': 0.6}
                    
                    # Calculate weekly ARPU and customer counts
                    monthly_arpu = (arpu_base[product]['Monthly'] * region_multipliers[region]) / 4.33
                    yearly_arpu = (arpu_base[product]['Yearly'] * region_multipliers[region]) / 4.33
                    
                    monthly_customers = int((monthly_revenue * 1000000) / monthly_arpu) if monthly_arpu > 0 else 0
                    yearly_customers = int((yearly_revenue * 1000000) / yearly_arpu) if yearly_arpu > 0 else 0
                    
                    # Get current trend type for events
                    current_trend_type = None
                    for phase in trend_generator.trends[combination_key]:
                        if phase['start_week'] <= total_week_number < phase['end_week']:
                            current_trend_type = phase['trend_type']
                            break
                    
                    event_name = f"{region}_{product}_{current_trend_type}" if current_trend_type else "baseline"
                    
                    # Create records for both Monthly and Yearly (separate rows)
                    for payment_plan in ['Monthly', 'Yearly']:
                        if payment_plan == 'Monthly':
                            revenue = monthly_revenue
                            customers = monthly_customers
                        else:
                            revenue = yearly_revenue
                            customers = yearly_customers
                        
                        records.append({
                            'Week_Start_Date': current_date.strftime('%m/%d/%Y'),
                            'Week_End_Date': week_end.strftime('%m/%d/%Y'),
                            'Quarter': quarter_str,
                            'Fiscal_Year': f'{year}',
                            'Quarter_Number': f'Q{quarter_num}',
                            'Week_Number_In_Quarter': week_in_quarter,
                            'Week_Number_Total': total_week_number + 1,
                            'Year': current_date.year,
                            'Month': current_date.month,
                            'Month_Name': current_date.strftime('%B'),
                            'Region': region,
                            'Product': product,
                            'Payment_Plan': payment_plan,
                            'Revenue_Millions': round(revenue, 6),
                            'Customer_Count': customers,
                            'Trend_Type': current_trend_type or 'baseline',
                            'Active_Events': event_name
                        })
            
            current_date = week_end + timedelta(days=1)
            week_in_quarter += 1
            total_week_number += 1
    
    return pd.DataFrame(records)

def scale_to_quarterly_targets(df):
    """Scale the generated data to match quarterly revenue targets"""
    df_scaled = df.copy()
    
    for quarter in quarterly_targets.keys():
        target_revenue = quarterly_targets[quarter]
        
        # Get current quarter's actual total
        quarter_mask = df_scaled['Quarter'] == quarter
        current_total = df_scaled[quarter_mask]['Revenue_Millions'].sum()
        
        if current_total > 0:
            # Calculate scaling factor
            scale_factor = target_revenue / current_total
            
            # Apply scaling to revenue column for this quarter
            df_scaled.loc[quarter_mask, 'Revenue_Millions'] *= scale_factor
            
            # Recalculate customer counts based on new revenue
            for idx in df_scaled[quarter_mask].index:
                revenue = df_scaled.loc[idx, 'Revenue_Millions']
                payment_plan = df_scaled.loc[idx, 'Payment_Plan']
                
                # Simplified customer calculation based on payment plan
                if payment_plan == 'Monthly':
                    customers = max(1, int(revenue * 50000))
                else:  # Yearly
                    customers = max(1, int(revenue * 5000))
                
                df_scaled.loc[idx, 'Customer_Count'] = customers
    
    return df_scaled

# Generate the cleaned data
print(f"\nDISTRIBUTION SETTINGS:")
print(f"   Subscription (by period with randomization): {SUBSCRIPTION_DISTRIBUTION}")
print(f"   Regional (by period): {REGIONAL_DISTRIBUTION}")
print(f"   Product distributions vary by region and time period")

# Step 1: Generate base data
df = generate_weekly_data()
print(f"Generated {len(df):,} records with region-specific trends")

# Step 2: Scale to match quarterly targets
print("Scaling data to match quarterly revenue targets...")
final_df = scale_to_quarterly_targets(df)

# Generate summary statistics
print("\n" + "="*60)
print("DATASET SUMMARY")
print("="*60)

print(f"Total Records: {len(final_df):,}")
print(f"Date Range: {final_df['Week_Start_Date'].min()} to {final_df['Week_End_Date'].max()}")
print(f"Total Weeks: {final_df['Week_Number_Total'].max()}")
print(f"Total Revenue: ${final_df['Revenue_Millions'].sum():,.2f}M")

# Revenue by payment plan
payment_summary = final_df.groupby('Payment_Plan')['Revenue_Millions'].sum()
print(f"\nREVENUE BY PAYMENT PLAN:")
for plan, revenue in payment_summary.items():
    pct = (revenue / final_df['Revenue_Millions'].sum()) * 100
    print(f"   {plan}: ${revenue:,.2f}M ({pct:.1f}%)")

print(f"Total Customers: {final_df['Customer_Count'].sum():,}")

print(f"\nREVENUE BY REGION:")
region_summary = final_df.groupby('Region')['Revenue_Millions'].sum().sort_values(ascending=False)
for region, revenue in region_summary.items():
    pct = (revenue / final_df['Revenue_Millions'].sum()) * 100
    print(f"   {region}: ${revenue:,.2f}M ({pct:.1f}%)")

print(f"\nREVENUE BY PRODUCT:")
product_summary = final_df.groupby('Product')['Revenue_Millions'].sum().sort_values(ascending=False)
for product, revenue in product_summary.items():
    pct = (revenue / final_df['Revenue_Millions'].sum()) * 100
    print(f"   {product}: ${revenue:,.2f}M ({pct:.1f}%)")

print(f"\nTARGET ACCURACY CHECK:")
quarterly_check = final_df.groupby('Quarter')['Revenue_Millions'].sum()
for quarter, target in list(quarterly_targets.items())[:10]:
    actual = quarterly_check.get(quarter, 0)
    error_pct = abs(actual - target) / target * 100
    status = "✅" if error_pct < 0.1 else "⚠️"
    print(f"{status} {quarter}: Target ${target}M, Actual ${actual:.1f}M (Error: {error_pct:.3f}%)")

print(f"\nTREND TYPES:")
trend_counts = final_df['Trend_Type'].value_counts()
for trend, count in trend_counts.items():
    print(f"   {trend}: {count:,} instances")

# Save the improved data
output_filename = 'zoom_improved_revenue_data1111.csv'
final_df.to_csv(output_filename, index=False)

print(f"File saved: {output_filename}")
print(f"Ready for Tableau analysis!")

# Display sample improved data
print(f"\nSAMPLE IMPROVED DATA PREVIEW:")
print("="*120)
sample_df = final_df[['Week_Start_Date', 'Quarter', 'Region', 'Product', 'Payment_Plan',
                     'Revenue_Millions', 'Customer_Count', 'Trend_Type']].head(20)
print(sample_df.to_string(index=False))

🚀 Generating IMPROVED revenue dataset...
🎛️  Using region-specific trends and time-aware distributions...
🎲 Added randomized subscription distribution (±3% variation)

📊 DISTRIBUTION SETTINGS:
   Subscription (by period with randomization): {'early': {'Monthly': 8, 'Yearly': 92}, 'mid': {'Monthly': 12, 'Yearly': 88}, 'recent': {'Monthly': 18, 'Yearly': 82}}
   Regional (by period): {'early': {'NA': 55, 'EMEA': 30, 'APAC': 10, 'LATAM': 5}, 'mid': {'NA': 48, 'EMEA': 35, 'APAC': 12, 'LATAM': 5}, 'recent': {'NA': 40, 'EMEA': 35, 'APAC': 18, 'LATAM': 7}}
   Product distributions vary by region and time period
✅ Generated 7,920 records with region-specific trends
📊 Scaling data to match quarterly revenue targets...

📋 IMPROVED DATASET SUMMARY
📅 Total Records: 7,920
📅 Date Range: 01/03/2020 to 12/26/2024
📅 Total Weeks: 330
💰 Total Revenue: $19,881.00M

💳 REVENUE BY PAYMENT PLAN:
   Monthly: $2,740.82M (13.8%)
   Yearly: $17,140.18M (86.2%)
👥 Total Customers: 222,737,809

🌍 REVENUE BY REGION:
